In [13]:
import numpy as np
import pandas as pd
from math import ceil
from itertools import product
from scipy.sparse import csr_matrix, csr_array, lil_matrix
import scipy.sparse as scs
import xarray as xr
import math
from functools import reduce

import gurobipy as gp
from gurobipy import GRB

import cvxpy as cp

In [2]:
import ps_data_worker as dw

In [6]:
RTS_DATA_DIR = 'RTS_Data'

In [7]:
# load rts data
rtsdata = dw.RTSDataSet(RTS_DATA_DIR)

# prepare model data
psdata = dw.create_ps_data_from_rts_data(rtsdata)
gencost = dw.create_cost_data_from_rts_data(rtsdata)

# preprare time series
first_period = (2020, 1, 1, 1) # year, month, day, period
horizon = 24 # #periods
gen_ts_mw, load_ts_mw = dw.prepare_da_timeseries_data(first_period, horizon, rtsdata, psdata)

>>> Reading system data
>>> Reading time series data


In [9]:
# prepare model data

basemva = rtsdata.basemva

buses = psdata.busdata
gens = psdata.gendata
branches = psdata.branchdata

# list of gens with timeseries dependent injctions
gens_with_ts = list(gen_ts_mw.keys())

# per unit load
D_pu = {}
for k,v in load_ts_mw.items():
    D_pu[k] = np.array(v)/basemva

# per unit ts generation
gen_ts_pu = {}
for k,v in gen_ts_mw.items():
    gen_ts_pu[k] = np.array(v)/basemva

Nbus = len(buses)
Ngen = len(gens)
Nbranch = len(branches)
Nt = horizon

# get slack bus
slack = [i for i,bus in enumerate(buses) if bus['is_slack']]
slack_ind = slack[0]

# compute ptdf (not currently used in model)
b_vec = np.array([1/branch['x_pu'] for branch in branches])
b_diag = csr_matrix(np.diag(b_vec))
lines_list = [i for i in range(Nbranch)]*2
fromtobus = [branch['from_bus'] for branch in branches] + [branch['to_bus'] for branch in branches]
data = [1]*Nbranch + [-1]*Nbranch 
inc_mat = csr_matrix((data, (lines_list, fromtobus)))
B_branch = b_diag @ inc_mat
B_bus = inc_mat.T @ B_branch
buses_sans_slack = [i for i in range(Nbus) if i not in slack]
B_bus_sans_slack = B_bus[buses_sans_slack][:,buses_sans_slack]
B_bus_inv_sans_slack = scs.linalg.inv(B_bus_sans_slack)
# add row/col of zeros before index slack
B_bus_pseudoinv = B_bus_inv_sans_slack
for i in slack:
    zero_row = i # insert row before old row of that index
    zero_col = i # insert col before old col of that index
    B_bus_pseudoinv._shape = (B_bus_pseudoinv._shape[0]+1, B_bus_pseudoinv._shape[1]+1)
    B_bus_pseudoinv.indices[B_bus_pseudoinv.indices >= zero_col] += 1
    B_bus_pseudoinv.indptr = np.insert(B_bus_pseudoinv.indptr, zero_row+1, B_bus_pseudoinv.indptr[zero_row])
ptdf = B_branch @ B_bus_pseudoinv
ptdf.data = np.round(ptdf.data, 6)
ptdf.eliminate_zeros()


In [10]:
# Basic SCUC

m = gp.Model()

# main variables
p = m.addVars(list(range(Ngen)), list(range(Nt)), lb=0, ub=GRB.INFINITY, name="p")
u = m.addVars(list(range(Ngen)), list(range(Nt)), vtype=GRB.BINARY, name="u") #offline indicator
v = m.addVars(list(range(Ngen)), list(range(Nt)), vtype=GRB.BINARY, name="v") #start-up indicator
w = m.addVars(list(range(Ngen)), list(range(Nt)), vtype=GRB.BINARY, name="w") #shut-down indicator
f = m.addVars(list(range(Nbranch)), list(range(Nt)), lb=-GRB.INFINITY, ub=GRB.INFINITY, name="f")
theta = m.addVars(list(range(Nbus)), list(range(Nt)), lb=-GRB.INFINITY, ub=GRB.INFINITY, name="theta")

consts = []
for t in range(Nt):
    for b,bus in enumerate(buses):
        # bus energy balance
        m.addConstr(
            sum(p[g,t] for g in bus['gens']) + 
            sum(f[l,t] for l in bus['branches_in']) - 
            sum(f[l,t] for l in bus['branches_out']) == D_pu[bus['id']][t]
        )
    
    # generator constraints
    for g,gen in enumerate(gens):
        if gen['id'] in gens_with_ts:
            m.addConstr(u[g,t] == 1)
            m.addConstr(v[g,t] == 0)
            m.addConstr(w[g,t] == 0)
            m.addConstr(p[g,t] <= gen_ts_pu[gen['id']][t])
        else:
            # production limits
            m.addConstr(p[g,t] <= gen['pmax_pu']*u[g,t])
            m.addConstr(p[g,t] >= gen['pmin_pu']*u[g,t])
            # ramping
            if t>0:
                m.addConstr(p[g,t] - p[g,t-1] <= gen['ramp_rate_pu_min']*60*u[g,t-1] + gen['pmin_pu']*v[g,t])
                m.addConstr(p[g,t-1] - p[g,t] <= gen['ramp_rate_pu_min']*60*u[g,t-1] + gen['pmin_pu']*w[t,i])
            # binary logic
            if t>0:
                m.addConstr(v[g,t] - w[g,t] == u[g,t] - u[g,t-1])
            else:
                m.addConstr(v[g,t] - w[g,t] == u[g,t])

    # power flow definition and constraints
    m.addConstr(theta[slack_ind, t] == 0)
    for l,line in enumerate(branches):
        # power flow model using voltage angles
        m.addConstr(f[l,t] == 1/line['x_pu'] * (theta[line['from_bus'],t] - theta[line['to_bus'],t]))
        m.addConstr(f[l,t] <= line['cap_pu'])
        m.addConstr(f[l,t] >= -line['cap_pu'])
                      
# objective with simplified production cost
production_cost = sum(
        sum(p[g,t]*gencost[g].pwlc.slopes[0] for g in range(Ngen))
    for t in range(Nt))
fixed_cost = sum(
        sum(v[g,t]*gencost[g].startup for g in range(Ngen)) +
        sum(w[g,t]*gencost[g].shutdown for g in range(Ngen))
    for t in range(Nt))
objective = production_cost + fixed_cost

m.setObjective(objective, GRB.MINIMIZE)

m.optimize()

Set parameter Username
Academic license - for non-commercial use only - expires 2025-01-30


In [14]:
# Basic SCUC cvxpy version

# main variables
p = cp.Variable((Ngen, Nt), pos=True, name="p")
u = cp.Variable((Ngen, Nt), boolean=True, name="u") #offline indicator
v = cp.Variable((Ngen, Nt), boolean=True, name="v") #start-up indicator
w = cp.Variable((Ngen, Nt), boolean=True, name="w") #shut-down indicator
f = cp.Variable((Nbranch, Nt), name="f")
theta = cp.Variable((Nbus, Nt), name="theta")

consts = []
for t in range(Nt):
    for b,bus in enumerate(buses):
        # bus energy balance
        consts.append(
            sum(p[g,t] for g in bus['gens']) + 
            sum(f[l,t] for l in bus['branches_in']) - 
            sum(f[l,t] for l in bus['branches_out']) == D_pu[bus['id']][t]
        )
    
    # generator constraints
    for g,gen in enumerate(gens):
        if gen['id'] in gens_with_ts:
            consts.append(u[g,t] == 1)
            consts.append(v[g,t] == 0)
            consts.append(w[g,t] == 0)
            consts.append(p[g,t] <= gen_ts_pu[gen['id']][t])
        else:
            consts.append(p[g,t] <= gen['pmax_pu']*u[g,t])
            consts.append(p[g,t] >= gen['pmin_pu']*u[g,t])
            if t>0:
                consts.append(p[g,t] - p[g,t-1] <= gen['ramp_rate_pu_min']*60*u[g,t-1] + gen['pmin_pu']*v[g,t])
                consts.append(p[g,t-1] - p[g,t] <= gen['ramp_rate_pu_min']*60*u[g,t-1] + gen['pmin_pu']*w[t,i])
            if t>0:
                consts.append(v[g,t] - w[g,t] == u[g,t] - u[g,t-1])
            else:
                consts.append(v[g,t] - w[g,t] == u[g,t])

    # power flow definition and constraints
    consts.append(theta[slack_ind, t] == 0)
    for l,line in enumerate(branches):
        # power flow model using voltage angles
        consts.append(f[l,t] == 1/line['x_pu'] * (theta[line['from_bus'],t] - theta[line['to_bus'],t]))
        consts.append(f[l,t] <= line['cap_pu'])
        consts.append(f[l,t] >= -line['cap_pu'])
                      
# objective with simplified production cost
production_cost = sum(
        sum(p[g,t]*gencost[g].pwlc.slopes[0] for g in range(Ngen))
    for t in range(Nt))
fixed_cost = sum(
        sum(v[g,t]*gencost[g].startup for g in range(Ngen)) +
        sum(w[g,t]*gencost[g].shutdown for g in range(Ngen))
    for t in range(Nt))
objective = production_cost + fixed_cost

theprob = cp.Problem(cp.Minimize(objective), consts)
theprob.solve(solver='GUROBI')

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/cvxpy/problems/problem.py:158: UserWarning: Objective contains too many subexpressions. Consider vectorizing your CVXPY code to speed up compilation.
  warnings.warn("Objective contains too many subexpressions. "


45080.00596252813